Pre-processamento da base de dados de apps do Google Play

Import das bibliotecas utilizadas

In [0]:
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

In [0]:
df = pd.read_csv('googleplaystore.csv')

Verificando as informações do DataFrame para entender os tipos dos atributos encontrar valores nulos, NaN e remove-los para que o uso dos algoritmos de IA seja possível.

In [0]:
df.head()

In [0]:
# Remover linhas com algum valor NaN

df.dropna(inplace = True)

Apartir dessa limpeza inicial de valores NaN, podemos continuar..

In [0]:
# # Limpeza da classificação de installs

df['Installs'] = [int(i[:-1].replace(',','')) for i in df['Installs']]

Convertendo os tipos de classificação do atributo que define o método de pagamento em binário

In [0]:
# Convertendo classificação da coluna Type em binário
def type_cat(types):
    if types == 'Free':
        return 1
    else:
        return 0

df['Type'] = df['Type'].map(type_cat)

Conversão da seção de classificação de conteúdo em números inteiros

In [0]:
# Categorizando a coluna Content Rating

le = preprocessing.LabelEncoder()
df['Content Rating'] = le.fit_transform(df['Content Rating'])

Percebemos que algumas informações não eram necessárias para um algoritmo de aprendizado de máquina

In [0]:
# Removendo informações desnecessárias

df.drop(labels = ['Last Updated','Current Ver','App'], axis = 1, inplace = True)

In [0]:
# Categorizando a coluna gênero

le = preprocessing.LabelEncoder()
df['Genres'] = le.fit_transform(df['Genres'])

In [0]:
# Categorizando a coluna Android Ver

le = preprocessing.LabelEncoder()
df['Android Ver'] = le.fit_transform(df['Android Ver'])

In [0]:
# Limpando a coluna Price

df['Price'] = df['Price'].apply(lambda x : x.strip('$'))

In [0]:
# Converter Reviews em valores inteiros

df['Reviews'] = df['Reviews'].astype(int)

In [0]:
# Codificação de variável fictícia para Categorias

category_list = df['Category'].unique().tolist() 
category_list = ['Category_' + word for word in category_list]
df = pd.concat([df, pd.get_dummies(df['Category'], prefix='Category')], axis=1)

In [0]:
# Dimensionamento e limpeza do tamanho do atributo de Size (números de instalações)

k_indices = df['Size'].loc[df['Size'].str.contains('k')].index.tolist()
converter = pd.DataFrame(df.loc[k_indices, 'Size'].apply(lambda x: x.strip('k')).astype(float).apply(lambda x: x / 1024).apply(lambda x: round(x, 3)).astype(str))
df.loc[k_indices,'Size'] = converter

df['Size'] = df['Size'].apply(lambda x: x.strip('M'))
df[df['Size'] == 'Varies with device'] = 0
df['Size'] = df['Size'].astype(float)

In [0]:
# Normalizar colunas necessárias
# (-1, 1) if your data has a single feature or array

scaler = MinMaxScaler()
tmp = np.array(df['Reviews'])
reviewsNormalized = scaler.fit_transform(df['Reviews'].astype('float').values.reshape(-1, 1))
df['Reviews'] = reviewsNormalized

In [0]:
sizeNormalized = scaler.fit_transform(df['Size'].astype('float').values.reshape(-1, 1))
df['Size'] = sizeNormalized

In [0]:
# Categorizando a coluna Installs

le = preprocessing.LabelEncoder()
df['Installs'] = le.fit_transform(df['Installs'])

In [0]:
# Pós normalização

df.head()

In [0]:
#  Separando o dataset tratado em features(X) e target(y)

features = ['Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres']

features.extend(category_list) # Adicionando as colunas dummies
X = df[features]
y = df['Rating']

knnModel = KNeighborsRegressor()

param_grid = { 'n_neighbors': [1, 3, 5, 7, 9, 21, 23, 25, 27, 43, 45, 47, 71, 73, 77] }

GS_knn = GridSearchCV(estimator = knnModel, param_grid = param_grid, cv = 5)
GS_knn.fit(X, y)
gs_df = pd.DataFrame(GS_knn.cv_results_)

x = gs_df['mean_test_score']
y = gs_df['param_n_neighbors']
plt.plot(y, x)
plt.title('Número n de vizinhos por taxa de acerto')
plt.xlabel('N vizinhos')
plt.ylabel('score')
plt.show()

In [0]:
X = df[features]
y = df['Rating']

rfrModel = RandomForestRegressor()

param_grid = { 'n_estimators': [5, 10, 15, 20, 30, 40, 50, 100, 200, 400] }

GS_rfr = GridSearchCV(estimator = rfrModel, param_grid = param_grid, cv = 5)
GS_rfr.fit(X, y)
gs_df = pd.DataFrame(GS_rfr.cv_results_)

x = gs_df['mean_test_score']
y = gs_df['param_n_estimators']
plt.plot(y, x)
plt.title('Número n de estimadores por taxa de acerto')
plt.xlabel('N estimadores')
plt.ylabel('score')
plt.show()

In [0]:
X = df[features]
y = df['Rating']

treeModel = DecisionTreeRegressor()

param_grid = { 'max_depth': [2, 4, 6, 8, 14, 22, 30, 36, 42, 56, 60, 68] }

GS_tree = GridSearchCV(estimator = treeModel, param_grid = param_grid, cv = 5)
GS_tree.fit(X, y)
gs_df = pd.DataFrame(GS_tree.cv_results_)

x = gs_df['mean_test_score']
y = gs_df['param_max_depth']
plt.plot(y, x)
plt.title('Número de níveis da árvore por taxa de acerto')
plt.xlabel('Profundidade máxima da árvore')
plt.ylabel('score')
plt.show()

In [0]:
X = df[features]
y = df['Rating']

svrModel = SVR(gamma='scale')

param_grid = { 'kernel': ['linear', 'rbf', 'poly', 'sigmoid', 'precomputed', 'callable'] }

GS_tree = GridSearchCV(estimator = svrModel, param_grid = param_grid, cv = 5)
GS_tree.fit(X, y)
gs_df = pd.DataFrame(GS_tree.cv_results_)

x = gs_df['mean_test_score']
y = gs_df['param_kernel']
plt.plot(y, x)
plt.title('Tipo de kernel por taxa de acerto')
plt.xlabel('kernel')
plt.ylabel('score')
plt.show()

In [0]:
# Inicializando modelos
knnModel = KNeighborsRegressor(n_neighbors = 15)
knnScores = []

svrModel = SVR(gamma='scale')
svrScores = []

decitionTreeModel = DecisionTreeRegressor()
decitionTreeScores = []

rfrModel = RandomForestRegressor(n_estimators=300)
rfrScores = []

# Separando em dado de treino e dado de teste, utilizando o KFold
kf = KFold(n_splits = 20, shuffle = True, random_state = 42)

for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    #knn
    knnModel.fit(X_train, y_train)
    knnScores.append(knnModel.score(X_test, y_test))

    #svr
    svrModel.fit(X_train, y_train)
    svrScores.append(svrModel.score(X_test, y_test))
    
    #decition tree
    decitionTreeModel.fit(X_train, y_train)
    decitionTreeScores.append(decitionTreeModel.score(X_test, y_test))

    #random forest regressor
    rfrModel.fit(X_train, y_train)
    rfrScores.append(rfrModel.score(X_test, y_test))
    

print('média k-nn: ', np.mean(knnScores))
print('média svr: ', np.mean(svrScores))
print('média decision trees: ', np.mean(decitionTreeScores))
print('média random forest: ', np.mean(rfrScores))